In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Prachi\\Documents\\Chicken-Disease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Prachi\\Documents\\Chicken-Disease-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallBacksConfig:
    root_dir : Path
    tensorboard_root_log_dir : Path
    checkpoint_model_filepath : Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories

In [7]:
from pathlib import Path

project_root = Path(r"C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project")

CONFIG_FILE_PATH = project_root / "config" / "config.yaml"
PARAMS_FILE_PATH = project_root / "params.yaml"

print("Config file path:", CONFIG_FILE_PATH)
print("Params file path:", PARAMS_FILE_PATH)

print("Config exists?", CONFIG_FILE_PATH.is_file())
print("Params exists?", PARAMS_FILE_PATH.is_file())


Config file path: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\config\config.yaml
Params file path: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\params.yaml
Config exists? True
Params exists? True


In [12]:
class ConfigurationManager:
    def __init__ (self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root], verbose=True)

    def get_prepare_callbacks_config (self) -> PrepareCallBacksConfig : 
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),
                            Path(config.tensorboard_root_log_dir)],verbose = True)
            
        prepare_callback_config = PrepareCallBacksConfig(
            root_dir= Path(config.root_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir)
        )
            
        return prepare_callback_config 

In [10]:
from pathlib import Path
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
from cnnClassifier import logger
import time


In [11]:
class PrepareCallbacks:
    def __init__ (self, config:PrepareCallBacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ] 

In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

✅ DEBUG YAML PATH: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\config\config.yaml
✅ DEBUG YAML CONTENT: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/PrachiVedant/datasets/raw/refs/heads/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}} <class 'dict'>
✅ DEBUG YAML PATH: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\params.yaml
✅ DEBUG YAML CONTENT: {'AUGMENTATI